# Pobranie Danych

dane zostały poprane ze strony [macrotrends.net](https://www.macrotrends.net/stocks) przy pomocy skryptu `scrape.py` i umieszczone kolejno w folderach `balance-sheet`, `cash-flow-statement`, `financial-ratios`, `income-statement`.
Kazdy folder posiada pliki csv z odpowiadającym raportem finansowym dla kazdej z firm indeksu S&P500. Firmy z indeksu S&P zostały pobrane z publicznego repozytrium github - [link](https://github.com/datasets/s-and-p-500-companies/blob/main/data/constituents.csv)

# Oczyszczanie danych i Transfomracje na danych

Zbiorcze dane z raportów finansowych (`balance-sheet`, `cash-flow-statement`, `financial-ratios`, `income-statement`) zostały umieszczone w oddzielnych plikach dla danego typu raportu i danej firmy. 
Dlatego przed przystąpieniem do analizy nalezy dane połączyć do zbiorczych plików CSV które będą zawierać raporty dla wszystkich firm z indeksu.

In [41]:
import pandas as pd
import glob
import os
from datetime import datetime


NOT_INCLUDE = [
    #'BF.B',
    #'BRK.B',
    'GOOG',
    'FOX',
    'NWS'
]

constituents = pd.read_csv('sp500/constituents.csv')

constituents['Date added'] = pd.to_datetime(constituents['Date added'])
constituents = constituents.loc[~constituents['Symbol'].isin(NOT_INCLUDE)] # remove classes other than A
selected_companies = set(constituents['Symbol'].to_list())

folders = [
    "financial-ratios",
    "balance-sheet",
    "income-statement",
    "cash-flow-statement"
]

df = None

for ticker in selected_companies:
    all_files_exists = all(
        [os.path.isfile(f"sp500/{folder}/{ticker}.csv") for folder in folders]
    )
    if not all_files_exists:
        continue

    ticker_df = None
    
    for folder in folders:
        file_path = f"sp500/{folder}/{ticker}.csv"
        new_df = pd.read_csv(file_path)
        new_df = new_df.rename(columns={'Unnamed: 0': 'date'})
        new_df['date'] = pd.to_datetime(new_df['date'])
        new_df.fillna(0, inplace=True)
        #new_df = new_df[(new_df['date'].dt.month.eq(12))]
        new_df['ticker'] = ticker
        
        new_df.set_index(['ticker', 'date'], inplace=True)
        
        if ticker_df is None:
            ticker_df = new_df
            continue

        ticker_df = ticker_df.merge(new_df, left_index=True, right_on=['ticker', 'date'])

    if df is None:
        df = ticker_df
        continue
    
    df = pd.concat([df, ticker_df])
len(df.reset_index()['ticker'].unique())


494

# Pobieranie danych dot. cen z yahoo finance

Tworzymy dataframe z cenami zamknięcie na koniec kwartałów Q1, Q2, Q3, Q4

In [43]:
import yfinance as yf
start_date = df.reset_index()['date'].min().strftime('%Y-%m-%d')
end_date = df.reset_index()['date'].max().strftime('%Y-%m-%d')
ticks = yf.Tickers(list(t.replace(".", "-") for t in selected_companies))
sp500_close = ticks.history(start=start_date, end=end_date, interval="1wk").Close
sp500_close_index = yf.Tickers('^GSPC').history(start=start_date, end=end_date, interval="1wk").Close
sp500_close = sp500_close.merge(sp500_close_index, left_index=True, right_on='Date')
sp500_close = sp500_close.rename(
    columns={"BRK-B": "BRK.B", "BF-B": "BF.B", "^GSPC": "S&P500"}
)

sp500_close = sp500_close.resample("ME").apply(lambda ser: ser.iloc[-1,])
_mask = (
    (sp500_close.index.month == 12)|
    (sp500_close.index.month == 9) |
    (sp500_close.index.month == 6) |
    (sp500_close.index.month == 3)
)
sp500_close = sp500_close.loc[_mask]
sp500_close.tail()

[*********************100%***********************]  500 of 500 completed
[*********************100%***********************]  1 of 1 completed


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,S&P500
Date,,,,,,,,,,,,,,,,,,,,,
2023-09-30,110.638901,170.151154,NaN,137.210007,94.432320,75.796120,301.002777,509.899994,171.361954,72.344803,...,29.587763,91.141502,54.642731,112.671181,89.742645,121.925407,110.800575,236.529999,171.862289,4288.049805
2023-12-31,137.838837,191.591370,NaN,136.139999,107.916412,70.623230,345.358246,596.599976,195.247864,69.704727,...,33.756351,90.121147,59.121479,96.682045,113.111710,128.110504,120.421608,273.329987,195.434387,4769.830078
2024-03-31,144.507294,170.861755,NaN,164.960007,111.974518,87.901062,342.371887,504.600006,195.407181,61.195770,...,35.215309,101.360130,52.318111,113.450340,128.202118,136.624924,130.854126,301.440002,167.930588,5254.350098
2024-06-30,128.945877,210.145279,NaN,151.630005,102.876785,95.936150,300.860596,555.539978,226.407120,59.377838,...,28.029408,88.967468,52.516781,113.269836,134.892838,131.166779,107.806511,308.929993,172.539764,5460.479980
2024-09-30,144.472549,226.550751,NaN,129.850006,112.111626,109.220207,360.771515,507.220001,227.267700,58.676655,...,32.676723,105.167198,63.206326,123.812614,133.097351,135.911438,104.213295,366.720001,189.580765,5751.069824


# Dodanie atrybutów decyzyjnych

* `log_retuns_q1`, `log_retuns_q2`, `log_retuns_q3`, `log_retuns_q4` - dodanie logarytmów ze zwrotów w przyszłych kwartałach - korzystam z logarytmów ponieważ operujemy w dziedzinie zwrotów, która jest multiplikatywna. Przez logarytmowanie zamieniamy multiplikatywność na addytywność. Więcej na temat dlaczego to robię zamieszczam w [linku do artykułu medium](https://medium.com/@manojkotary/simple-returns-vs-log-returns-a-comprehensive-comparative-analysis-for-financial-analysis-702403693bad)
* dodatkowo definiuję klasy binarne: 
    - `class_1_q1`, `class_1_q2`, `class_1_q3`, `class_1_q4` - 1 - jeśli `log_return_qX` > 0 - tzn. jeśli spółka zanotuje wzrost
    - `class_2_q1`, `class_2_q2`, `class_2_q3`, `class_2_q4` - 1 - jeśli `log_return_qX` > log_return_qX(SP&500) - tzn. jeśli spółka zanotuje wzrost większy niż S&P500
    - `diff_q1`, `diff_q2`, `diff_q3`, `diff_q4` - różnica pomiędzy zwrotami z S&P500 - lepsze w ocenie korelacji

In [53]:
import numpy as np


HORIZON = 4

def calculate_log_returns(df, shift=1):
    returns_df = df.copy()
    returns_df = np.log(df / df.shift(shift))
    returns_df = returns_df.dropna(how='all')
    return returns_df


log_returns = [None] * (HORIZON + 1)


for i in range(1, HORIZON + 1):
    log_returns[i] = pd.melt(
        calculate_log_returns(sp500_close, i).reset_index(),
        id_vars=['Date'],
        value_vars=list(sp500_close.columns),
        var_name='ticker',
        value_name=f'log_return_q{i}'
    ).rename(columns={"Date": 'date'}).set_index(['date', 'ticker']).sort_index()


target = pd.melt(
    sp500_close.reset_index(),
    id_vars=['Date'],
    value_vars=list(sp500_close.columns),
    var_name='ticker',
    value_name='price'
).rename(columns={"Date": 'date'}).set_index(['date', 'ticker']).sort_index()




for i in range(1, HORIZON + 1):
    target[f'log_return_q{i}'] = log_returns[i][f'log_return_q{i}']
    target[f'class_1_q{i}'] = (target[f'log_return_q{i}'] > 0) * 1

for i in range(len(target)):
    idx = target.index[i]
    spx_idx = (idx[0], 'S&P500')

    for i in range(1, HORIZON + 1):
        spx_return = target.at[spx_idx, f'log_return_q{i}']
        stock_return = target.at[idx, f'log_return_q{i}']
        target.at[idx, f'diff_q{i}'] = stock_return - spx_return
        target.at[idx, f'class_2_q{i}'] = (stock_return > spx_return) * 1

for i in range(1, HORIZON + 1):
    target[f'class_2_q{i}'] = target[f'class_2_q{i}'].astype('int')

target = target.dropna(subset = ['log_return_q1'])
target.to_csv('sp500/target.csv')

In [54]:
columns =  ['price'] + [f'log_return_q{i}'for i in range(1, HORIZON + 1)] + [f'class_1_q{i}'for i in range(1, HORIZON + 1)]  +  [f'class_2_q{i}'for i in range(1, HORIZON + 1)] +  [f'diff_q{i}'for i in range(1, HORIZON + 1)]
target = target.reindex(columns, axis=1)
target.to_csv('sp500/target.csv')

In [55]:
t = target.reset_index()
t.loc[t['ticker'] == 'AAPL']

,date,ticker,price,log_return_q1,log_return_q2,log_return_q3,log_return_q4,class_1_q1,class_1_q2,class_1_q3,class_1_q4,class_2_q1,class_2_q2,class_2_q3,class_2_q4,diff_q1,diff_q2,diff_q3,diff_q4
1,2009-06-30,AAPL,4.218338,0.188281,NaN,NaN,NaN,1,0,0,0,1,0,0,0,0.126246,NaN,NaN,NaN
427,2009-09-30,AAPL,5.570424,0.278030,0.466311,NaN,NaN,1,1,0,0,1,1,0,0,0.143786,0.270032,NaN,NaN
853,2009-12-31,AAPL,6.348595,0.130762,0.408792,0.597074,NaN,1,1,1,0,1,1,1,0,0.046716,0.190502,0.316748,NaN
1280,2010-03-31,AAPL,7.108994,0.113127,0.243890,0.521920,0.710201,1,1,1,1,1,1,1,1,0.058168,0.104884,0.248670,0.374916
1710,2010-06-30,AAPL,7.439487,0.045441,0.158568,0.289331,0.567361,1,1,1,1,1,1,1,1,0.187015,0.245184,0.291899,0.435686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26677,2023-09-30,AAPL,170.151154,-0.123463,0.040283,0.280152,0.220104,0,1,1,1,0,0,1,1,-0.086306,-0.002294,0.169662,0.041203
27171,2023-12-31,AAPL,191.591370,0.118678,-0.004786,0.158961,0.398830,1,0,1,1,1,0,1,1,0.012199,-0.074107,0.009905,0.181861
27665,2024-03-31,AAPL,170.861755,-0.114510,0.004168,-0.119296,0.044451,0,1,0,1,0,0,0,0,-0.211256,-0.199057,-0.285362,-0.201350
28160,2024-06-30,AAPL,210.145279,0.206944,0.092434,0.211112,0.087649,1,1,1,1,1,0,0,0,0.168464,-0.042792,-0.030593,-0.116899


## Transformacje na danych

* zamiana kwartału na integer - dla uproszczeń
* wypełnienie brakujących atrybutów zerami
* eliminacja firm z krótką historią publikacji tzn. poniżej 20 kwartałów

In [56]:
Y = pd.read_csv('sp500/target.csv')
Y['date'] = pd.to_datetime(Y['date'])
start_date = Y['date'].min()

Y['quarter'] = Y['date'].apply(lambda x: 4*(x.year - 2009) + x.quarter - 1)
Y = Y.reset_index()
Y = Y.set_index(keys=['quarter', 'ticker']).sort_index()

X = pd.read_csv('sp500/data.csv')
X['date'] = pd.to_datetime(X['date'])
X = X.loc[X['date'] >= start_date]
X['quarter'] = X['date'].apply(lambda x: 4*(x.year - 2009) + x.quarter - 1)
X = X.set_index(keys=['quarter', 'ticker']).sort_index()


# Some companies are publishing different parameters that are relevant for industry 
# ex. gross_margin is not popular reported value in bank sector because is not important for that kind of company
# I decided to make fill nans with 0.0 instead of deleting them and find useful parameters after correlation survey
X = X.fillna(value=0.0) 

X = X.drop(columns=['date'])
Y = Y.drop(columns=['date'])
df = X.join(Y)

# We need companies with at least 4 years of history
df = df.reset_index()
quarter_counts = df.groupby('ticker')['quarter'].nunique()
valid_tickers = quarter_counts[quarter_counts >= 20].index
df = df.loc[df['ticker'].isin(valid_tickers)]
df = df.set_index(keys=['quarter', 'ticker']).sort_index()


features = X.columns.to_list()
target_features = Y.columns.to_list()

## Wzbogacanie danych o kwartalne zmiany

Dodaję zmiany atrybutów na przestrzeni kwartałów korzystając z funkcji `diff`. Robię to w celu dodania informacji na temat dynamiki zmian atrybutów. Akcje tak jak inne aktywa wpadają w trendy, które mają potencjał kształotwania cen w przyszłości.

In [57]:
new_features = []

for feature in features:
    for n in [2, 4, 8, 12, 16 ,20]:
        new_feature = f"{feature}_{n}_quartes_gain"
        df[new_feature] = df.groupby("ticker")[feature].diff(periods=n).fillna(value=0)
        new_features.append(new_feature)

features = list(set(df.columns) - set(target_features))

/var/folders/tx/5nr7htks60x_hwx8s_wsynxc0000gn/T/ipykernel_66275/2584920762.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_feature] = df.groupby("ticker")[feature].diff(periods=n).fillna(value=0)
/var/folders/tx/5nr7htks60x_hwx8s_wsynxc0000gn/T/ipykernel_66275/2584920762.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_feature] = df.groupby("ticker")[feature].diff(periods=n).fillna(value=0)
/var/folders/tx/5nr7htks60x_hwx8s_wsynxc0000gn/T/ipykernel_66275/2584920762.py:6: PerformanceWarning: DataFrame is hig

## Wzbogacanie danych przez dodanie percentylów w stosunku do reszty rynku

Dodaje infomracje nt tego jak niektóre parametry finansowe wyglądają na tle indeksu i sektora w jakim działają poszczególne firmy

In [58]:
import numpy as np
from tqdm import tqdm


def clean_ratio(values):
    return np.array([x for x in values if np.isfinite(x) and x > 0])


def analyze_percentile(clean_ratios, target_pe):
    percentile = np.percentile(clean_ratios, np.linspace(0, 100, 101))
    target_percentile = np.interp(target_pe, percentile, np.linspace(0, 100, 101))
    return target_percentile

financial_ratios_features = set(pd.read_csv('sp500/financial-ratios/AAPL.csv').columns.to_list()[1:]).intersection(set(df.columns))


NOT_INCLUDE = [
    #'BF.B',
    #'BRK.B',
    'GOOG',
    'FOX',
    'NWS'
]

constituents = pd.read_csv('sp500/constituents.csv')
sectors = constituents['GICS Sector'].unique()
constituents['Date added'] = pd.to_datetime(constituents['Date added'])
constituents = constituents.loc[~constituents['Symbol'].isin(NOT_INCLUDE)] # remove classes other than A


dff = df.reset_index()

print(financial_ratios_features)
for q in tqdm(dff['quarter'].unique()):
    for sector in sectors:
        sector_companies = constituents.loc[constituents['GICS Sector'] == sector]['Symbol'].to_list()
        df_all = dff.loc[dff['quarter'] == q]
        df_sector = df_all.loc[df_all['ticker'].isin(sector_companies)]
        for feat in financial_ratios_features:
            cr_all = clean_ratio(df_all[feat].values)
            cr_sector = clean_ratio(df_sector[feat].values)
            for ticker in sector_companies:
                try:
                    val = df.loc[(q, ticker), feat].to_numpy()[0]
                    if len(cr_all):
                        percentile_in_index = analyze_percentile(cr_all, val)
                        df.loc[(q, ticker), f'{feat}_percentile_in_index'] = percentile_in_index
                    
                    if len(cr_sector):
                        percentile_in_sector = analyze_percentile(cr_sector, val)
                        df.loc[(q, ticker), f'{feat}_percentile_in_sector'] = percentile_in_sector
                    
                except KeyError:
                    pass

{'days_sales_in_receivables', 'free_cash_flow_per_share', 'asset_turnover', 'gross_margin', 'inventory_turnover', 'roa', 'long_term_debt_capital', 'ebit_margin', 'operating_margin', 'net_profit_margin', 'current_ratio', 'book_value_per_share', 'receiveable_turnover', 'return_on_tangible_equity', 'roe', 'pre_tax_profit_margin', 'debt_equity_ratio', 'roi', 'operating_cash_flow_per_share'}


  0%|          | 0/63 [00:00<?, ?it/s]

/var/folders/tx/5nr7htks60x_hwx8s_wsynxc0000gn/T/ipykernel_66275/494172546.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[(q, ticker), f'{feat}_percentile_in_index'] = percentile_in_index
/var/folders/tx/5nr7htks60x_hwx8s_wsynxc0000gn/T/ipykernel_66275/494172546.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[(q, ticker), f'{feat}_percentile_in_sector'] = percentile_in_sector
/var/folders/tx/5nr7htks60x_hwx8s_wsynxc0000gn/T/ipykernel_66275/494172546.py:47: PerformanceWarning: DataFrame is highly fragment

## Dodanie Market Cap

Dodaję brakujące kapitalizację spółek - z uwagi na to że to ważny parametr. Usuwam rekordy bez kapitalizacji.

In [61]:
import math

df['market_cap'] = df['price']*df['net_income']/df['eps_basic_net_earnings_per_share']

def recompute_market_cap(basic_shares_outstanding, price, market_cap):
    if not math.isnan(basic_shares_outstanding) and basic_shares_outstanding > 0.0:
        return price * basic_shares_outstanding
    return market_cap

df['market_cap'] = df[['basic_shares_outstanding', 'price', 'market_cap']].apply(lambda x : recompute_market_cap(*x), axis=1)


In [62]:
df = df.dropna(subset=['price'])
df = df.dropna(subset=['market_cap'])
df.dropna(subset=['price']).to_csv("sp500/data_xy.csv")